In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install huggingface

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -q transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from transformers import BertModel 
from datasets import load_dataset


model = BertModel.from_pretrained("bert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa")
dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")

In [6]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [8]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [9]:
#dataset
dataset['train'][0]

{'ID': '2017-En-21441',
 'Tweet': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'anger': False,
 'anticipation': True,
 'disgust': False,
 'fear': False,
 'joy': False,
 'love': False,
 'optimism': True,
 'pessimism': False,
 'sadness': False,
 'surprise': False,
 'trust': True}

In [10]:
labels = [label for label in dataset['train'].features.keys() if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'love',
 'optimism',
 'pessimism',
 'sadness',
 'surprise',
 'trust']

Now we are getting ready to preprocess the data using the BERT tokenizer. This include mapping the text to float ing point labels and moving it into a matrix of size batch_size x num_labels. These should be floats per PyTorch expectation.

In [11]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    #get the text of this batch
    text=examples["Tweet"]
    #get the ecoding for this text tusing bert tokenizer
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length = 128)
    #add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    #numpy array size batch x num labels
    labels_matrix = np.zeros((len(text), len(labels)))

    # replace zeros in numpy array with values from encoding
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [12]:
text = dataset['train'][5]['Tweet']
len(text)

94

In [13]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [14]:
dataset['train'][0]['Tweet']

"“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry"

In [15]:
example = encoded_dataset['train'][0]
print(example.keys())

len(example['input_ids'])
example['labels']

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]

In [16]:
tokenizer.decode(example['input_ids'])
#CLS = classify token and it is placed at the beginning of input
#SEP = end of string used for next sentence prediction
#PAD = pad to 128 characters

"[CLS] “ worry is a down payment on a problem you may never have '. joyce meyer. # motivation # leadership # worry [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [17]:
example['labels']

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]

In [18]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['anticipation', 'optimism', 'trust']

In [19]:
encoded_dataset.set_format("torch")

Setup model. multi_label_classification indicates the type of problem. We'll use BCEWithLogitsLoss (sigmoid layer with binary cross entropy loss - BCEWithLogitsLoss)

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
batch_size = 8
metric_name = "f1"
from transformers import TrainingArguments, Trainer

In [22]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

/Users/alexandertekle/Library/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [25]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [42]:
encoded_dataset['train'][5]

{'input_ids': tensor([  101,  2053,  2021,  2008,  1005,  1055,  2061, 10140,  1012,  2012,
          6342,  2001,  2763, 11004,  2055,  7760,  2077,  2021,  9115,  3271,
          2014,  2041,  1057, 16050,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [47]:
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), 
                labels=encoded_dataset['train'][0]['labels'].unsqueeze(0),
                attention_mask=encoded_dataset['train'][0]['attention_mask'].unsqueeze(0))

In [48]:
outputs

SequenceClassifierOutput(loss=tensor(0.6801, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.1945, -0.0671, -0.3757, -0.1490,  0.0881, -0.0456,  0.2394,  0.2131,
         -0.0378,  0.1188,  0.2153]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [46]:
outputs

SequenceClassifierOutput(loss=tensor(0.7598, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.3129, -0.6629, -0.6483,  0.3060,  0.0961, -0.0388, -0.1145,  0.3261,
         -0.1480,  0.2126,  0.0373]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)